##### En este notebook  hago Análisis exploratorio de datos sobre las ventas de un producto en particular y el comportamiento de los socios que lo consumieron.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

In [ ]:
df= pd.read_csv(r'C:\Users\Pablo\OneDrive - Helacor S.A\Club Grido\Extractos DW\CustomerID compraron S-TACC.csv')
df.head()

In [ ]:
socios_STACC = df.CustomerId.count()
print('Socios que compraron productos sin TACC:', socios_STACC)

In [ ]:
print('minima fecha:', df['fecha primera compra SIN TACC'].min())
print('maxima fecha:', df['fecha primera compra SIN TACC'].max())

In [ ]:
ventas= pd.read_csv(r'C:\Users\Pablo\OneDrive - Helacor S.A\Club Grido\Extractos DW\ventas totales Customer con TACC.csv')
ventas.head()

In [ ]:
ventas_STACC= ventas[ventas.ProductId.isin([242,243,244,245,246,247])]

In [ ]:
ventas_STACC['Sabor'] = np.where(ventas_STACC['ProductName'].str.contains('Frutilla'),'Frutilla',np.where(ventas_STACC['ProductName'].str.contains('Frutos'),'Frutos del bosque','Mango y Maracuya'))
ventas_STACC['Cantidad'] = np.where(ventas_STACC['ProductName'].str.contains('6'),'Pack','Individual')
ventas_STACC.head(100)

In [ ]:
ventas_STACC['mes'] = pd.to_datetime(ventas_STACC['SaleDate']).dt.month
ventas_STACC['año'] = pd.to_datetime(ventas_STACC['SaleDate']).dt.year

In [ ]:
X= ventas_STACC.groupby(["año","mes"],as_index=False)["ProductWeightGrams"].sum()
X.columns= ['año','mes','Kilos']
X['Kilos'] = (X['Kilos']/1000).apply(lambda x: round(x,1))
X["mes-año"] = X["mes"].astype(str)+"-" + X["año"].astype(str)

fig, ax = plt.subplots(figsize=(15,5))
ax.tick_params(axis='x', labelrotation = 30)
fig = sns.lineplot(data= X, x= "mes-año",y= "Kilos")
#fig.show()

In [ ]:
X

In [ ]:
kg_STACC = int(ventas_STACC.ProductWeightGrams.sum()/1000)
print('Ventas totales de productos sin TACC:', kg_STACC)

In [ ]:
venta_agrupada= pd.DataFrame(ventas_STACC.groupby('ProductName')['ProductWeightGrams'].sum()).reset_index()
venta_agrupada["Sabor"] = np.where(venta_agrupada['ProductName'].str.contains('Frutilla'),'Frutilla',np.where(venta_agrupada['ProductName'].str.contains('Frutos'),'Frutos del bosque','Mango y Maracuya'))
venta_agrupada.head()

In [ ]:
grupo_producto = pd.DataFrame(venta_agrupada.groupby(venta_agrupada['ProductName']).ProductWeightGrams.sum()/venta_agrupada.ProductWeightGrams.sum()*100).apply(lambda x: round(x, 1)).reset_index()
grupo_producto.columns= ['Producto','% Kilos']
grupo_producto.sort_values('% Kilos',ascending=False)

In [ ]:
grupo_sabor = pd.DataFrame(venta_agrupada.groupby(venta_agrupada['Sabor']).ProductWeightGrams.sum()/venta_agrupada.ProductWeightGrams.sum()*100).apply(lambda x: round(x, 1)).reset_index()
grupo_sabor.columns= ['Sabor','% Kilos']
grupo_sabor.sort_values('% Kilos',ascending=False)

Se podria continuar haciendo un analisis sobre cómo se venden los productos segun la forma. Los individuales son impulsivos para acompañar otros productos? que productos?

#### Ahora, hago analisis por CustomerId. Compró anteriormente? cuanto? se asocio gracias a S/TACC? Cambio la frecuencia de consumo?

In [ ]:
ventas = ventas.merge(df[['CustomerId','fecha primera compra SIN TACC']],on='CustomerId')

In [ ]:
ventas.head()

In [ ]:
ventas['compra anterior'] = np.where(ventas['SaleDate']< ventas['fecha primera compra SIN TACC'],1,0)
#ventas['compra anterior'] = np.where(ventas['SaleDate'] == ventas['fecha primera compra SIN TACC'],-1,ventas['compra anterior'])

In [ ]:
ventas_anteriores = ventas[ventas['compra anterior'] ==1]
ventas_posteriores = ventas[ventas['compra anterior'] ==0]

In [ ]:
ventas.shape

In [ ]:
ventas_anteriores.shape

In [ ]:
ventas_posteriores.shape

In [ ]:
frec_compra_antes = ventas_anteriores.groupby('CustomerId').agg({'SaleId':'count','SaleDate':['min','max'],'ProductWeightGrams':'sum'}).reset_index()
frec_compra_antes.columns=['CustomerId','Cantidad de compras antes','Primera compra historica','ultima compra previo a compra S/TACC','kg antes']
frec_compra_antes['kg antes']=frec_compra_antes['kg antes']/1000
frec_compra_antes

In [ ]:
frec_compra_dps = ventas_posteriores.groupby('CustomerId').agg({'SaleId':'count','SaleDate':'max','ProductWeightGrams':'sum'}).reset_index()
frec_compra_dps.columns=['CustomerId','Cantidad de compras dps','Ultima compra historica','kg dps']
frec_compra_dps['kg dps']=frec_compra_dps['kg dps']/1000
frec_compra_dps

In [ ]:
df= df.merge(frec_compra_antes,on="CustomerId",how='left')
df= df.merge(frec_compra_dps,on="CustomerId",how='left')

In [ ]:
df.head()

### Inactividad

In [ ]:
print('Socios cuya primera compra registrada incluye producto Sin TACC:',df['ultima compra previo a compra S/TACC'].isnull().sum())
print('% de socios cuya primera compra registrada incluye producto Sin TACC sobre el total de socios que compraron Sin TACC:',round(df['ultima compra previo a compra S/TACC'].isnull().sum()/df.shape[0]*100,1))

In [ ]:
df['Dias inactivo previo a compra S/TACC'] = abs(pd.to_datetime(df['fecha primera compra SIN TACC']) - pd.to_datetime(df['ultima compra previo a compra S/TACC'])).apply(lambda x: x.days)

In [ ]:
clusters = [0,30,90,np.inf]
nombres_clusters =["menor a 30 días","entre 30 y 90 días","mas de 90 días"]
df['Inactividad previa'] = pd.cut(df['Dias inactivo previo a compra S/TACC'],bins= clusters, labels = nombres_clusters)
df['Inactividad previa']= np.where(df['Inactividad previa'].isnull(),'socio nuevo',df['Inactividad previa'])
df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.tick_params(axis='x', labelrotation = 30)
plt.title('Inactividad de socios previo a la primera compra de producto S/TACC')
fig = sns.countplot(data=df, y="Inactividad previa", order = ["menor a 30 días","entre 30 y 90 días","mas de 90 días","socio nuevo"])
ax.set_xlabel('Cantidad de socios',loc='center')

In [ ]:
plt.hist(df["Dias inactivo previo a compra S/TACC"],bins = 500)
plt.xlim(0,30)
plt.xlabel("Dias inactivo previo a compra de producto S/TACC")
plt.ylabel("Cantidad de apariciones")

In [ ]:
clusters_2 = [0,90,np.inf]
nombres_clusters_2 =["menor a 90 días","más de 90 días o nuevo"]
inactividad = pd.cut(df['Dias inactivo previo a compra S/TACC'],bins= clusters_2, labels = nombres_clusters_2)
inactividad= np.where(inactividad.isnull(),'más de 90 días o nuevo',inactividad)

fig, ax = plt.subplots(figsize=(10,5))
ax.tick_params(axis='x', labelrotation = 30)
plt.title('Inactividad de socios previo a la primera compra de producto S/TACC')
fig = sns.countplot(y= inactividad)
ax.set_xlabel('Cantidad de socios',loc='center')

In [ ]:
unique, counts = np.unique(inactividad, return_counts=True)
dict(zip(unique,counts))

### Frecuencia de compra

In [ ]:
df['Frecuencia de compra antes'] = (pd.to_datetime(df['fecha primera compra SIN TACC']) - \
                                    pd.to_datetime(df['Primera compra historica'])).apply(lambda x: x.days)\
                                    / df['Cantidad de compras antes']

# En lugar de ultima compra historica, va 29/09/2021
df["fecha_fin"]='2021-09-29 16:36:43.9586853 +00:00'
df["fecha_fin"] = pd.to_datetime(df["fecha_fin"])
df['Frecuencia de compra dps'] = (df.fecha_fin -pd.to_datetime(df['fecha primera compra SIN TACC'])).apply(lambda x: x.days)/df["Cantidad de compras dps"]
df.drop(columns="fecha_fin",inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df[~df["Cantidad de compras antes"].isnull()]["Frecuencia de compra antes"].mean()

In [ ]:
df[~df["Cantidad de compras antes"].isnull()]["Frecuencia de compra dps"].mean()

In [ ]:
df["cambio frecuencia de compra"] =  df["Cantidad de compras antes"] - df["Frecuencia de compra dps"]

In [ ]:
df["cambio frecuencia de compra"].mean()

In [ ]:
plt.hist(df["cambio frecuencia de compra"],bins=1000)
plt.xlim(-100,100)
plt.xlabel("Frecuencia de compra antes - Frecuencia de compra despues")
plt.ylabel("Cantidad de apariciones")
plt.title('Cambio en frecuencias de compra')

In [ ]:
df.groupby('Inactividad previa').agg({"cambio frecuencia de compra":'mean'}).apply(lambda x: round(x,1))

### Ticket promedio

In [ ]:
df["Ticket promedio antes"] = df["kg antes"] /df["Cantidad de compras antes"]
df["Ticket promedio dps"] = df["kg dps"] / df["Cantidad de compras dps"]
df["Cambio en ticket promedio"] = df["Ticket promedio dps"] - df["Ticket promedio antes"]

In [ ]:
plt.hist(df["Cambio en ticket promedio"],bins=1000)
plt.xlim(-1,1)
plt.xlabel("Ticket promedio despues - Ticket promedio antes")
plt.ylabel("Cantidad de apariciones")
plt.title('Cambio en Ticket promedio')

In [ ]:
df["Cambio en ticket promedio"].mean()

In [ ]:
df["Cambio en ticket promedio"].median()

In [ ]:
df["Ticket promedio antes"].mean()

In [ ]:
df["Ticket promedio dps"].mean()

In [ ]:
print('% de gente que empeoró su ticket promedio:',df[df["Cambio en ticket promedio"]<0].shape[0]/df.shape[0]*100)

In [ ]:
print("Variacion en ticket promedio desde 11/20 hasta 9/21: ", round((0.45/0.51 -1)*100,1),"%")

In [ ]:
print("Variacion en ticket promedio despues de comprar sin TACC ", round((df["Ticket promedio dps"].mean()/df["Ticket promedio antes"].mean() - 1)*100,1),"%")

In [ ]:
variacion_ticket_promedio = df.groupby('Inactividad previa').agg({"Ticket promedio antes":'mean',"Ticket promedio dps":'mean'})
variacion_ticket_promedio['Variacion'] = (variacion_ticket_promedio["Ticket promedio dps"]/variacion_ticket_promedio["Ticket promedio antes"]- 1).apply(lambda x: round(x*100,1))
variacion_ticket_promedio

In [ ]:
ventas_ticket_prom = ventas.groupby(["SaleId","SaleDate"]).agg({"Amount":'sum',"ProductWeightGrams":'sum'})
ventas_ticket_prom['Kilos']= (ventas_ticket_prom["ProductWeightGrams"]/1000)
ventas_ticket_prom.reset_index(inplace=True)
ventas_ticket_prom['SaleDate'] = pd.to_datetime(ventas_ticket_prom['SaleDate'])
ventas_ticket_prom.set_index('SaleDate')

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(ventas_ticket_prom.groupby(pd.Grouper(key='SaleDate',freq='M'))['Kilos'].mean())

### Cross - selling

Armo mi master de productos

In [ ]:
productos = pd.DataFrame(ventas.groupby(['ProductId','ProductName','ProductWeightGrams']).Amount.count()).reset_index()
productos.drop('Amount',axis=1,inplace=True)

In [ ]:
productos.drop_duplicates(subset='ProductId',inplace=True)

Separo todas las ventas que hayan incluido productos sin tacc y tambien los demas productos

In [ ]:
toda_ventas_STACC= pd.DataFrame(ventas[ventas.SaleId.isin(ventas_STACC.SaleId)].groupby(['SaleId','ProductId']).Amount.count()).reset_index()
toda_ventas_STACC.head()

In [ ]:
total_ventas = toda_ventas_STACC.groupby('SaleId').count().shape[0]
print('Ventas totales sin TACC:', total_ventas)

In [ ]:
medicion_cantidades= toda_ventas_STACC.groupby('SaleId').count()
ventas_unicas = medicion_cantidades[medicion_cantidades.Amount==1].shape[0]
print('De todas las ventas en las que hubo un producto sin TACC, el', round(ventas_unicas/total_ventas*100,1),"% fueron solas")

In [ ]:
ventas_acompañadas = medicion_cantidades[medicion_cantidades.Amount>1].shape[0]
print('De todas las ventas en las que hubo un producto sin TACC, el', round(ventas_acompañadas/total_ventas*100,1),"% fueron junto a otro producto")

#### Analizo las ventas que incluyeron solo Yogurt sin TACC en pack

In [ ]:
ventas_pack = ventas[ventas['SaleId'].isin(ventas[ventas.ProductId.isin([245,246,247])].SaleId)]
ventas_pack= ventas_pack[~ventas_pack.ProductId.isin([245,246,247])]

total_ventas_pack= ventas_pack.groupby('SaleId').Amount.count().shape[0]


pack_productos = pd.DataFrame(ventas_pack.ProductId.value_counts().reset_index())
pack_productos.columns = ['ProductId','Cantidad apariciones']
pack_productos= pack_productos.merge(productos,on='ProductId')
pack_productos["% participacion ventas"] = (pack_productos["Cantidad apariciones"]/total_ventas_pack*100).apply(lambda x: round(x,1))
pack_productos.head(20)

In [ ]:
ventas[ventas.ProductId.isin([245,246,247])].SaleId

#### Ahora, analizo las ventas que no incluyeron otro producto sin TACC

In [ ]:
medicion_cantidades_STACC =toda_ventas_STACC[~toda_ventas_STACC.ProductId.isin([242,243,244,245,246,247])].groupby('SaleId').count()

In [ ]:
ventas_acompañadas_STACC = medicion_cantidades_STACC.shape[0]
print('De todas las ventas en las que hubo un producto sin TACC, el', round(ventas_acompañadas_STACC/total_ventas*100,1),"% fueron junto a otro producto")

In [ ]:
productos_asociados = pd.DataFrame(toda_ventas_STACC[~toda_ventas_STACC.ProductId.isin([242,243,244,245,246,247])].ProductId.value_counts()).reset_index()
productos_asociados.columns = ['ProductId','Cantidad apariciones']
productos_asociados= productos_asociados.merge(productos,on='ProductId')
productos_asociados["% participacion ventas"] = (productos_asociados["Cantidad apariciones"]/total_ventas*100).apply(lambda x: round(x,1))
productos_asociados.head(20)

### Recompra de clientes

In [ ]:
recompra = pd.DataFrame(ventas[ventas['compra anterior']==0].groupby('CustomerId').agg({'SaleId':'nunique','SaleDate':'min'})).reset_index()
print( 'El ', round(recompra[recompra.SaleId > 1].shape[0]/ socios_STACC*100,1),'% de los socios que compró un producto sin TACC, volvió a comprar en la cadena')

In [ ]:
recompra_STACC = pd.DataFrame(ventas_STACC.groupby('CustomerId').SaleId.nunique()).reset_index()
total_recompra = recompra_STACC.shape[0]
print('El',round(recompra_STACC[recompra_STACC.SaleId > 1].shape[0] / socios_STACC * 100,1),' % de los clientes, volvieron a comprar productos sin TACC')

In [ ]:
recompra = recompra.merge(df[['CustomerId','fecha primera compra SIN TACC']],on='CustomerId')
#recompra ['Días para la recompra'] = recompra['fecha primera compra SIN TACC'] - recompra['SaleDate']
recompra

In [ ]:
recompra ['Días para la recompra'] = pd.to_datetime(recompra['SaleDate']) - pd.to_datetime(recompra['fecha primera compra SIN TACC']) 
recompra

In [ ]:
print('Promedio días para la recompra:', recompra['Días para la recompra'].mean())